## Level: Medium

#### 题目来源：力扣（LeetCode） 链接：https://leetcode-cn.com/problemset/lcof/?difficulty=%E4%B8%AD%E7%AD%89

## Part I：数组

### 1. 机器人的运动范围

#### 地上有一个m行n列的方格，从坐标 [0,0] 到坐标 [m-1,n-1] 。一个机器人从坐标 [0, 0] 的格子开始移动。
#### 它每次可以向左、右、上、下移动一格（不能移动到方格外），也不能进入行坐标和列坐标的数位之和大于k的格子。
#### 例如，当k为18时，机器人能够进入方格 [35, 37] ，因为3+5+3+7=18。但它不能进入方格 [35, 38]，因为3+5+3+8=19。
#### 请问该机器人能够到达多少个格子？

#### 思路：BFS in matrix. 利用Queue实现BFS

#### 求`x`数为和：`x % 10`, `x // 10`, 循环求得数位和

#### 隐藏的优化：搜索方向可以从上、下、左、右缩减为向右和向下

In [3]:
def movingCount(m, n, k):
    
    directions = [(0,1), (1,0)]
    
    queue = [(0,0)]
    visited = set()
    
    while queue:
        current = queue.pop()
        visited.add(current)
        for _x, _y in directions:
            x_new = current[0] + _x
            y_new = current[1] + _y
            if isValid(m, n, k, x_new, y_new, visited):
                queue.append((x_new, y_new))
    
    return len(visited)


def num_sum(num):
    
    res = 0
    
    while num:
        res += num % 10
        num = num // 10
    
    return res 


def isValid(m, n, k, x, y, visited):
    
    if 0 <= x < m and 0 <= y < n and num_sum(x) + num_sum(y) <= k and (x,y) not in visited:
        return True 
    return False

In [6]:
m = 2 
n = 3
k = 1
movingCount(m, n, k)

3

### 2. 股票的最大利润

#### 假设把某股票的价格按照时间先后顺序存储在数组中，请问买卖该股票一次可能获得的最大利润是多少？

#### 思路：DP

#### `dp[i]` 以`prices[i]`结尾的子数组得最大利润，即前`i`日的最大利润

#### 初始状态：$dp[0] = 0$
#### 状态转移：$dp[i] = max(dp[i-1], prices[i] - min(prices[0:i]))$

- 在遍历`prices`数组时，可以借助一个变量`cost`来更新最低价格

#### 返回值：$dp[n-1]$

In [7]:
def maxProfit(prices):
    
    if not prices:
        return 0 
    
    dp = [0] * len(prices)
    cost = prices[0]
    
    for i in range(1, len(prices)):
        
        dp[i] = max(dp[i-1], prices[i] - cost)
        
        cost = min(cost, prices[i])
    
    return dp[-1]


In [8]:
prices = [7, 1, 5, 3, 6]
maxProfit(prices)

5

In [18]:
### 空间优化

import math

def maxProfit(prices):
    
    dp = 0
    cost = math.inf
    
    for price in prices:
        
        cost = min(cost, price)
        
        dp = max(dp, price - cost)
            
    return dp

In [19]:
prices = [7, 1, 5, 3, 6]
maxProfit(prices)

5

### 3. 剪绳子

#### 给你一根长度为 n 的绳子，请把绳子剪成整数长度的 m 段（m、n都是整数，n>1并且m>1），
#### 每段绳子的长度记为 $k[0],k[1]...k[m-1]$ 。请问 $k[0]*k[1]*...*k[m-1]$ 可能的最大乘积是多少？
#### 例如，当绳子的长度是8时，我们把它剪成长度分别为2、3、3的三段，此时得到的最大乘积是18。


#### 思路：贪心

| 绳子切分方案        |    乘积     |       结论        |
|--------------------|-------------|-------------------|
|           1        |     1       |      NA           |
|     2 = 1 + 1      |  1 * 1 = 1  |    2不应该切分    |
|     3 = 1 + 2      |  1 * 2 = 2  |    3不应该切分    |
|  4 = 2 + 2 = 1 + 3 |  4 > 3      |    4切成2 + 2     |
|  5 = 2 + 3 = 1 + 4 |  6 > 4      |    5切成2 + 3     |
|  6 = 3 + 3 = 2 + 4 |  9 > 8      |    6切成3 + 3     |
|        ···         |   ···       |       ···        |

#### 从上表可以看出： 3比2优

- 以长度3为单位切割

- 最后若剩下1， 则与前一个3组合成 2 + 2 

- 最后若剩下2， 则不变

In [20]:
def cuttingRope(n):
    
    if n == 2:
        return 1 
    
    if n == 3:
        return 2 
    
    a = n % 3 
    b = n // 3
    
    if a == 0:
        return pow(3, b)
    
    if a == 1:
        return pow(3, b - 1) * 4
    
    if a == 2:
        return pow(3, b) * 2

In [21]:
n = 10
cuttingRope(n)

36

### 4. 把数组排成最小的数

#### 输入一个正整数数组，把数组里所有数字拼接起来排成一个数，打印能拼接出的所有数字中最小的一个。

#### 思路：利用快速排序算法自定义排序

#### 将`nums`中的数字按照如下规则从小到大排序：

- e.g `3`和`34`. 因为`334 < 343`, 所以`3` 排在`34`前面

- 按这样的规则从小到大排列之后得到的数字为拼接数字中最小的。

#### 如何实现这样的排序：

- 先将`nums`中的数字转成str，从而可以直接利用str的拼接和大小比较

- 利用新的规则来自定义快速排序

In [22]:
def minNumber(nums):
    
    numbers = [str(num) for num in nums]
    
    results = quickSort(numbers)
    
    return ''.join(results)



def quickSort(numbers):
    
    d = [[], [], []]
    
    pivot = numbers[len(numbers)//2]
    
    for num in numbers:
        if num + pivot < pivot + num:
            d[0].append(num)
        elif num + pivot > pivot + num:
            d[2].append(num)
        else:
            d[1].append(num)
    
    if len(d[0]) > 1:
        d[0] = quickSort(d[0])
        
    if len(d[2]) > 1:
        d[2] = quickSort(d[2])
    
    return d[0] + d[1] + d[2]

In [23]:
nums = [3,30,34,5,9]
minNumber(nums)

'3033459'

### 5. 丑数

#### 我们把只包含因子 2、3 和 5 的数称作丑数（Ugly Number）。求按从小到大的顺序的第 n 个丑数。

说明：1是丑数

#### 思路：DP

#### 丑数 = 某较小丑数 $\times$ 某因子( 2, 3, 5)

$$\left\{\begin{aligned}
x_a \times 2, a \in [1, n] \\
x_b \times 3, b \in [1, n] \\
x_c \times 5, c \in [1, n]
\end{aligned}
\right.$$

$$x_{n+1} = min(x_a \times 2, x_b \times 3, x_c \times 5)$$

$$\left\{\begin{aligned}
x_a \times 2 > x_n \ge x_{a-1} \times 2 \\
x_b \times 3 > x_n \ge x_{b-1} \times 3  \\
x_c \times 5 > x_n \ge x_{c-1} \times 5 
\end{aligned}
\right.$$

#### 每轮update$x_i$后要update$a, b, c$的位置

In [24]:
def nthUglyNumber(n):
    
    dp = [1] * n
    
    a = b = c = 0
    
    for i in range(1, n):
        
        na = 2*dp[a]
        nb = 3*dp[b]
        nc = 5*dp[c]
        
        dp[i] = min(na, nb, nc)
        
        if dp[i] == na:
            a += 1
        if dp[i] == nb:
            b += 1
        if dp[i] == nc:
            c += 1 
    
    return dp[-1]

In [25]:
n = 10
nthUglyNumber(n)

12

### 6. 求 $1 + 2 + ... + n$

#### 要求不能使用乘除法、for、while、if、else、switch、case等关键字及条件判断语句（A?B:C）

### 7. 数字序列中某一位的数字

#### 数字以0123456789101112131415…的格式序列化到一个字符序列中。
#### 在这个序列中，第5位（从下标0开始计数）是5，第13位是1，第19位是4，等等。

#### 请写一个函数，求任意第n位对应的数字


### 8. 1 ~ n整数中1出现的次数

#### 输入一个整数 n ，求1～n这n个整数的十进制表示中1出现的次数。

#### 例如，输入12，1～12这些整数中包含1 的数字有1、10、11和12，1一共出现了5次。

### 9. 礼物的最大价值

#### 在一个 m*n 的棋盘的每一格都放有一个礼物，每个礼物都有一定的价值（价值大于 0）。
#### 你可以从棋盘的左上角开始拿格子里的礼物，并每次向右或者向下移动一格、直到到达棋盘的右下角。
#### 给定一个棋盘及其上面的礼物的价值，请计算你最多能拿到多少价值的礼物？


### 10. 数组中数字出现的次数

#### 一个整型数组 `nums` 里除两个数字之外，其他数字都出现了两次。
#### 请写程序找出这两个只出现一次的数字。要求时间复杂度是O(n)，空间复杂度是O(1)。

### 11. 数组中数字出现的次数 II

#### 在一个数组 nums 中除一个数字只出现一次之外，其他数字都出现了三次。
#### 请找出那个只出现一次的数字。

### 12. 矩阵中的路径

#### 请设计一个函数，用来判断在一个矩阵中是否存在一条包含某字符串所有字符的路径。
#### 路径可以从矩阵中的任意一格开始，每一步可以在矩阵中向左、右、上、下移动一格。
#### 如果一条路径经过了矩阵的某一格，那么该路径不能再次进入该格子。

例如，在下面的3×4的矩阵中包含一条字符串“bfce”的路径。

[["a","b","c","e"],

 ["s","f","c","s"],

["a","d","e","e"]]


但矩阵中不包含字符串“abfb”的路径，因为字符串的第一个字符b占据了矩阵中的第一行第二个格子之后，路径不能再次进入这个格子。

### 13. 数值的整数次方

#### 实现函数`double Power(double base, int exponent)`，求`base`的`exponent`次方。
#### 不得使用库函数，同时不需要考虑大数问题。

## Part II：字符串

### 1. 字符串的排列

#### 输入一个字符串，打印出该字符串中字符的所有排列。

### 2. 最长不含重复字符的子字符串

#### 请从字符串中找出一个最长的不包含重复字符的子字符串，计算该最长子字符串的长度。

### 3. 把数字翻译成字符串

#### 给定一个数字，我们按照如下规则把它翻译为字符串：
#### 0 翻译成 “a” ，1 翻译成 “b”，……，11 翻译成 “l”，……，25 翻译成 “z”。
#### 一个数字可能有多个翻译。请编程实现一个函数，用来计算一个数字有多少种不同的翻译方法。

### 4. 把字符串转换成整数

#### 写一个函数 StrToInt，实现把字符串转换成整数这个功能。不能使用 atoi 或者其他类似的库函数。

首先，该函数会根据需要丢弃无用的开头空格字符，直到寻找到第一个非空格的字符为止。

当我们寻找到的第一个非空字符为正或者负号时，则将该符号与之后面尽可能多的连续数字组合起来，作为该整数的正负号；假如第一个非空字符是数字，则直接将其与之后连续的数字字符组合起来，形成整数。

该字符串除了有效的整数部分之后也可能会存在多余的字符，这些字符可以被忽略，它们对于函数不应该造成影响。

注意：假如该字符串中的第一个非空格字符不是一个有效整数字符、字符串为空或字符串仅包含空白字符时，则你的函数不需要进行转换。

在任何情况下，若函数不能进行有效的转换时，请返回 0。

说明：

假设我们的环境只能存储 32 位大小的有符号整数，那么其数值范围为 [−231,  231 − 1]。如果数值超过这个范围，请返回  INT_MAX (231 − 1) 或 INT_MIN (−231) 。


### 5. 表示数值的字符串

#### 请实现一个函数用来判断字符串是否表示数值（包括整数和小数）。
#### 例如，字符串"+100"、"5e2"、"-123"、"3.1416"、"0123"都表示数值，
#### 但"12e"、"1a3.14"、"1.2.3"、"+-5"、"-1E-16"及"12e+5.4"都不是。


## Part III: 栈和队列

### 1. 队列的最大值

#### 请定义一个队列并实现函数 `max_value` 得到队列里的最大值。
#### 要求函数`max_value`、`push_back` 和 `pop_front` 的均摊时间复杂度都是O(1)。

#### 若队列为空，`pop_front `和 `max_value` 需要返回 -1


### 2. 栈的压入、弹出序列

#### 输入两个整数序列，第一个序列表示栈的压入顺序，请判断第二个序列是否为该栈的弹出顺序。
#### 假设压入栈的所有数字均不相等。
#### 例如，序列 {1,2,3,4,5} 是某栈的压栈序列，序列 {4,5,3,2,1} 是该压栈序列对应的一个弹出序列，
#### 但 {4,3,5,1,2} 就不可能是该压栈序列的弹出序列。


## Part V: 树

In [1]:
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, x):
#         self.val = x
#         self.left = None
#         self.right = None

### 1. 二叉树中和为某一值的路径

#### 输入一棵二叉树和一个整数，打印出二叉树中节点值的和为输入整数的所有路径。
#### 从树的根节点开始往下一直到叶节点所经过的节点形成一条路径。

### 2. 重建二叉树

#### 输入某二叉树的前序遍历和中序遍历的结果，请重建该二叉树。
#### 假设输入的前序遍历和中序遍历的结果中都不含重复的数字。

### 3. 二叉搜索树与双向链表

#### 输入一棵二叉搜索树，将该二叉搜索树转换成一个排序的循环双向链表。
#### 要求不能创建任何新的节点，只能调整树中节点指针的指向。

### 4. 树的子结构

#### 输入两棵二叉树A和B，判断B是不是A的子结构。(约定空树不是任意一个树的子结构)

#### B是A的子结构， 即 A中有出现和B相同的结构和节点值。

### 5. 二叉搜索树的后序遍历序列

#### 输入一个整数数组，判断该数组是不是某二叉搜索树的后序遍历结果。
#### 如果是则返回 true，否则返回 false。假设输入的数组的任意两个数字都互不相同。

### 6. 从上到下打印二叉树 II

#### 从上到下打印出二叉树的每个节点，同一层的节点按照从左到右的顺序打印。

### 7. 从上到下打印二叉树 III

#### 请实现一个函数按照之字形顺序打印二叉树，
#### 即第一行按照从左到右的顺序打印，第二层按照从右到左的顺序打印，第三行再按照从左到右的顺序打印，
#### 其他行以此类推。

## Part VI: 链表

### 1. 复杂链表的复制

#### 请实现 `copyRandomList` 函数，复制一个复杂链表。
#### 在复杂链表中，每个节点除了有一个 `next` 指针指向下一个节点，还有一个 `random` 指针指向链表中的任意节点或者` null`。


In [4]:
# Definition for a Node.
# class Node:
#     def __init__(self, x: int, next: 'Node' = None, random: 'Node' = None):
#         self.val = int(x)
#         self.next = next
#         self.random = random